## Import Data

In [ ]:
import pandas as pd
import gdown
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora
from pprint import pprint

from gensim.models.coherencemodel import CoherenceModel


file_id = "1u7rvNf84a__E3HaPfhcC83MLENOieBZe"
url = f'https://drive.google.com/uc?id={file_id}'
output = 'data.csv'
gdown.download(url, output, quiet=False)
df = pd.read_csv('data.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Downloading...
From: https://drive.google.com/uc?id=1u7rvNf84a__E3HaPfhcC83MLENOieBZe
To: /content/data.csv
100%|██████████| 1.31G/1.31G [00:12<00:00, 101MB/s] 


### View the Data

In [ ]:
df

,DocID,Title,Abstract,BodyText
0,S0001457513002972,A system of safety management practices and wo...,Objective The overall research objective was t...,"In particular, the individual worker interface..."
1,S0001457513004806,Network-level accident-mapping: Distance based...,The objective of an accident-mapping algorithm...,The location coordinates of the accident is de...
2,S0001457514003091,Measuring errors and violations on the road: A...,The Driver Behavior Questionnaire (DBQ) is a s...,"Briefly, the Cohort II study randomly sampled ..."
3,S0001457515001098,Operating under the influence: Three year reci...,Operating a motor vehicle under the influence ...,"In addition to loss of life, the economic cost..."
4,S000145751500127X,Real-time driver drowsiness feedback improves ...,Driver drowsiness has been implicated as a maj...,"However, the effects of feedback on other appr..."
...,...,...,...,...
40086,S8756328219302856,PYY is a negative regulator of bone mass and s...,Objective: Bone loss in anorexia nervosa and f...,"Scans were performed at 50 kV, 200 μA, 0.5 mm ..."
40087,S8756328219304004,Development of protocols for the first serial ...,There is an unmet need for a high-resolution t...,We have generated and compared SBF SEM data fr...
40088,S8756328219304715,Overexpression of Pitx1 attenuates the senesce...,To explore the role of low expression of Pitx1...,"However, this process needs to be confirmed. C..."
40089,S8756328219304739,"The effect of pubertal timing, as reflected by...",Objective: To examine the relationship between...,"In terms of limitations, whereas timing of the..."


## Data Cleaning

In [ ]:
#create a copy which we will prepare differently from a gpt approach. We take only the first 1000 papers in this case to avoid colab crashing issues.
lda_df = df.head(1000).copy()

# Remove punctuation and lowercase
lda_df['BodyText'] = lda_df['BodyText'].map(lambda x: re.sub('[,\.!?]', '', x))
lda_df['BodyText'] = lda_df['BodyText'].map(lambda x: x.lower())

#Remove stop words as well as customed defined vocabulary (words I don't think add much information but show up frequently)
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'et', 'al', 'using', 'also', 'fig', 'study', 'may', 'however', 'two', 'one', 'used', 'within', 'data', 'used',
                   'time', 'different', 'analysis', 'model', 'table', 'studies', 'high', 'effect', 'results', 'signficant', 'found', 'based', 'observed', 'could', 'well',
                   'number', 'effects', 'first', 'eg', 'shown', 'associated', 'reported', 'compared', 'three', 'sample', 'would', 'similar', 'see', 'individuals', 'higher', 'due',
                   'non', 'min', 'low', 'increased', 'therefore', 'significant', 'range', 'related', 'change', 'mean', 'mm', 'thus', 'performed', 'likely', 'potential', 'specific',
                   'size', 'hz', 'showed', 'levels', 'models', 'ms'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]


data = lda_df.BodyText.values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

## Make the LDA Model

In [ ]:
# number of topics
num_topics = 50

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations=50)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(40,
  '0.004*"species" + 0.003*"cells" + 0.002*"cell" + 0.002*"mice" + '
  '0.002*"group" + 0.002*"protein" + 0.002*"samples" + 0.002*"control" + '
  '0.002*"including" + 0.001*"expression"'),
 (29,
  '0.003*"cells" + 0.002*"protein" + 0.002*"group" + 0.002*"cell" + '
  '0.002*"level" + 0.002*"participants" + 0.002*"species" + 0.001*"values" + '
  '0.001*"treatment" + 0.001*"method"'),
 (1,
  '0.002*"cell" + 0.002*"cells" + 0.002*"protein" + 0.002*"participants" + '
  '0.002*"level" + 0.002*"activity" + 0.002*"species" + 0.002*"patients" + '
  '0.002*"including" + 0.001*"although"'),
 (27,
  '0.003*"cells" + 0.002*"protein" + 0.002*"control" + 0.002*"cell" + '
  '0.002*"figure" + 0.002*"participants" + 0.002*"test" + 0.002*"function" + '
  '0.002*"species" + 0.001*"process"'),
 (45,
  '0.002*"participants" + 0.002*"samples" + 0.002*"cells" + 0.002*"protein" + '
  '0.002*"information" + 0.001*"structure" + 0.001*"energy" + 0.001*"factors" '
  '+ 0.001*"level" + 0.001*"control"'),
 (44

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('nCoherence Score: ', coherence_lda)


nCoherence Score:  0.23275802816420865


In [ ]:
!pip install funcy
!pip install tzdata
!pip install --no-deps pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 30.9 MB/s eta 0:00:00


In [ ]:
lda_model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
lda_viz = gensimvis.prepare(lda_model, corpus, id2word)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Final Result: Visualize the topics

In [ ]:
# You can select a topic by clicking a topic bubble, or typing the topic number in the "Selected Topic" box.
#The lambda slider is a custom metric created by LDAvis
#lambda = 1 is normal relevance, i.e., how frequently the word appears in that topics
#lambda = 0 is lift, i.e., likelihood in the topic divided by liklihood in the corpus (this will highlight words that are unique to this topic)

#We could begin to determine which topics represent which families by using lambda = 0
#check topic 8 for instance, given apatite, fluoroapatite and chloroapatite come up first,
#this topic assuredly deals with clay, and is likely about geogology
#topic 43 appears to deal with wind energy and sound, etc.
lda_viz

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.006332 -0.013602       1        1  14.120515
12    -0.010988 -0.010749       2        1   4.512569
30     0.003283  0.014088       3        1   3.801149
46     0.013250 -0.011198       4        1   3.757189
16    -0.000050 -0.002494       5        1   3.695373
41     0.002194  0.010243       6        1   3.071693
26     0.002361  0.003678       7        1   3.054055
24     0.016393 -0.002384       8        1   2.962412
42    -0.009908 -0.006453       9        1   2.837200
39     0.008116  0.002165      10        1   2.679242
32     0.008967  0.004984      11        1   2.652241
1     -0.005677 -0.000241      12        1   2.439717
3     -0.010703  0.009702      13        1   2.427119
31     0.003695 -0.005727      14        1   2.320206
21     0.000919 -0.000983      15        1   2.122163
5     -0.002348 -0.001888      16        1   2.025762
17     0.004480  0.000614      17        1   1.902928
7      0.002495  0.000873      18        1   1.864994
45     0.006976  0.006485      19        1   1.835205
47     0.000896 -0.003058      20        1   1.772118
36     0.001039  0.003193      21        1   1.738046
44     0.000723 -0.000013      22        1   1.708972
20    -0.003831  0.007074      23        1   1.693178
10     0.002615 -0.002477      24        1   1.568564
48    -0.003146 -0.002058      25        1   1.529180
40    -0.009958  0.000043      26        1   1.498383
49     0.001210 -0.001798      27        1   1.438836
18     0.004824 -0.000869      28        1   1.364555
25    -0.004255  0.003929      29        1   1.334814
33    -0.000659  0.004709      30        1   1.313284
14    -0.001782 -0.002600      31        1   1.276634
37     0.002484  0.000296      32        1   1.240332
11    -0.011180  0.006836      33        1   1.163776
27    -0.006432 -0.001559      34        1   1.135682
34     0.007294 -0.003449      35        1   1.120096
9     -0.000294 -0.004398      36        1   1.119711
2     -0.002879  0.002534      37        1   1.113198
22     0.007254 -0.009025      38        1   1.065820
29     0.001583  0.002864      39        1   1.064602
19    -0.002451  0.004985      40        1   1.020744
28    -0.003381  0.001702      41        1   0.982761
6     -0.001220 -0.000203      42        1   0.894975
0     -0.001305 -0.009918      43        1   0.868441
15    -0.000407  0.000347      44        1   0.828070
43    -0.002411 -0.005541      45        1   0.786173
38    -0.001691  0.000846      46        1   0.785178
8      0.000914 -0.004154      47        1   0.757519
23    -0.007492  0.001124      48        1   0.722187
35     0.006898  0.008885      49        1   0.577204
4     -0.000086  0.004640      50        1   0.435238, topic_info=               Term         Freq        Total Category  logprob  loglift
2960          cells  5805.000000  5805.000000  Default  30.0000  30.0000
8177        species  5385.000000  5385.000000  Default  29.0000  29.0000
5295           cell  4158.000000  4158.000000  Default  28.0000  28.0000
875    participants  4698.000000  4698.000000  Default  27.0000  27.0000
545           group  4584.000000  4584.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
11689       protein    14.130847  3923.938382  Topic50  -6.6254  -0.1895
875    participants    14.698498  4698.172284  Topic50  -6.5860  -0.3302
276         control    14.361190  4968.992816  Topic50  -6.6092  -0.4094
712           level    12.776055  3336.818312  Topic50  -6.7261  -0.1282
1068       response    12.486861  3074.941545  Topic50  -6.7490  -0.0693

[4378 rows x 6 columns], token_table=       Topic      Freq Term
term                       
50010      4  0.175587  aad
50010      9  0.175587  aad
42623      1  0.073431  aao
42623      2  0.048954  aao
42623      3  0.061193  aao
...      ...       ...  ...
52408      